## Python for NLP: Word Embeddings for Deep Learning in Keras

In this article, we will study word embeddings for NLP tasks that involve deep learning. We will see how word embeddings can be used to perform simple classification task using deep neural network in Python's Keras Library.

## Problems with One-Hot Encoded Feature Vector Approaches

A potential drawback with one-hot encoded feature vector approaches such as N-Grams, bag of words and TF-IDF approach is that the feature vector for each document can be huge. For instance, if you have a half million unique words in your corpus and you want to represent a sentence that contains 10 words, your feature vector will be a half million dimensional one-hot encoded vector where only 10 indexes will have 1. This is a wastage of space and increases algorithm complexity exponentially resulting in the curse of dimentionality.

## Word Embeddings
In word embeddings, every word is represented as an n-dimensional dense vector. The words that are similar will have similar vector. Word embeddings techniques such as GloVe and Word2Vec have proven to be extremely efficient for converting words into corresponding dense vectors. The vector size is small and none of the indexes in the vector is actually empty.

## Implementing Word Embeddings with Keras Sequential Models
The Keras library is one of the most famous and commonly used deep learning libraries for Python that is built on top of TensorFlow.

Keras support two types of APIs: Sequential and Functional. In this section we will see how word embeddings are used with Keras Sequential API. In the next section, I will explain how to implement the same model via the Keras functional API.

To implement word embeddings, the Keras library contains a layer called Embedding(). The embedding layer is implemented in the form of a class in Keras and is normally used as a first layer in the sequential model for NLP tasks.

The embedding layer can be used to peform three tasks in Keras:

* It can be used to learn word embeddings and save the resulting model
* It can be used to learn the word embeddings in addition to performing the NLP tasks such as text classification, sentiment analysis, etc.
* It can be used to load pretrained word embeddings and use them in a new model

In this article, we will see the second and third use-case of the Embedding layer. The first use-case is a subset of the second use-case.

Let's see how the embedding layer looks:

`embedding_layer = Embedding(200, 32, input_length=50)`

The first parameter in the embeddig layer is the size of the vocabulary or the total number of unique words in a corpus. The second parameter is the number of the dimensions for each word vector. For instance, if you want each word vector to have 32 dimensions, you will specify 32 as the second parameter. And finally, the third parameter is the length of the input sentence.

The output of the word embedding is a 2D vector where words are represented in rows, whereas their corresponding dimensions are presented in columns. Finally, if you wish to directly connect your word embedding layer with a densely connected layer, you first have to flatten your 2D word embeddings into 1D. These concepts will become more understandable once we see word embedding in action.

## Custom Word Embeddings

As I said earlier, Keras can be used to either learn custom words embedding or it can be used to load pretrained word embeddings. In this section, we will see how the Keras Embedding Layer can be used to learn custom word embeddings.

We will perform simple text classification tasks that will use word embeddings. Execute the following script to download the required libraries:

In [25]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense, Flatten
from tensorflow.keras.layers import Embedding

Next, we need to define our dataset. We will be using a very simple custom dataset that will contain reviews above movies. The following script creates our dataset:

In [3]:
corpus = [
    # Positive Reviews

    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews

    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]

Our corpus has 8 positive reviews and 8 negative reviews. The next step is to create label set for our data.

In [4]:
sentiments = array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

You can see that the first 8 items in the sentiment array contain 1, which correspond to positive sentiment. The last 8 items are zero that correspond to negative sentiment.

Earlier we said that the first parameter to the Embedding() layer is the vocabulary, or number of unique words in the corpus. Let's first find the total number of words in our corpus:

In [12]:
from nltk.tokenize import word_tokenize

all_words = []
for sent in corpus:
    tokenize_word = word_tokenize(sent)
    all_words = all_words + tokenize_word

In the script above, we simply iterate through each sentence in our corpus and then tokenize the sentence into words. Next, we iterate through the list of all the words and append the words into the all_words list. Once you execute the above script, you should see all the words in the all_words dictionary. However, we do not want the duplicate words.

We can retrieve all the unique words from a list by passing the list into the set function, as shown below.

In [16]:
unique_words = set(all_words)
print(len(unique_words))
vocab_length = len(unique_words) + 5

45


In the output you will see "45", which is the number of unique words in our corpus. We will add a buffer of 5 to our vocabulary size and will set the value of vocab_length to 50.

The Embedding layer expects the words to be in numeric form. Therefore, we need to convert the sentences in our corpus to numbers. One way to convert text to numbers is by using the one_hot function from the keras.preprocessing.text library. The function takes sentence and the total length of the vocabulary and returns the sentence in numeric form.

In [17]:
embedded_sentences = [ one_hot(sent, vocab_length) for sent in corpus]
print(embedded_sentences )

[[38, 32, 24, 41, 48], [39, 38, 31, 39, 48, 41, 13], [25, 30, 34, 13, 32, 18], [30, 44], [28, 47, 18, 38, 48, 41, 13], [19, 13, 39, 8], [4, 20, 32, 13, 35, 48], [13, 32, 13, 6, 48], [11, 12], [43, 19, 32], [6, 47], [13, 31, 23, 49], [48, 40, 25, 41, 39, 48], [39, 48, 31, 11], [48, 27, 25, 48], [39, 12, 32, 6]]


In the script above, we convert all the sentences in our corpus to their numeric form and display them on the console. The output looks like this:

You can see that our first sentence contained five words, therefore we have five integers in the first list item. Also, notice that the last word of the first sentence was "movie" in the first list item, and we have digit 48 at the fifth place of the resulting 2D array, which means that "movie" has been encoded as 48 and so on.

The embedding layer expects sentences to be of equal size. However, our encoded sentences are of different sizes. One way to make all the sentences of uniform size is to increase the lenght of all the sentences and make it equal to the length of the largest sentence. Let's first find the largest sentence in our corpus and then we will increase the length of all the sentences to the length of the largest sentence. To do so, execute the following script:

In [18]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

In the sentence above, we use a lambda expression to find the length of all the sentences. We then use the max function to return the longest sentence. Finally the longest sentence is tokenized into words and the number of words are counted using the len function.

Next to make all the sentences of equal size, we will add zeros to the empty indexes that will be created as a result of increasing the sentence length. To append the zeros at the end of the sentencses, we can use the pad_sequences method. The `first parameter is the list of encoded sentences of unequal sizes, the second parameter is the size of the longest sentence or the padding index, while the last parameter is padding where you specify post to add padding at the end of sentences.`

Execute the following script:

In [20]:
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences)

[[38 32 24 41 48  0  0]
 [39 38 31 39 48 41 13]
 [25 30 34 13 32 18  0]
 [30 44  0  0  0  0  0]
 [28 47 18 38 48 41 13]
 [19 13 39  8  0  0  0]
 [ 4 20 32 13 35 48  0]
 [13 32 13  6 48  0  0]
 [11 12  0  0  0  0  0]
 [43 19 32  0  0  0  0]
 [ 6 47  0  0  0  0  0]
 [13 31 23 49  0  0  0]
 [48 40 25 41 39 48  0]
 [39 48 31 11  0  0  0]
 [48 27 25 48  0  0  0]
 [39 12 32  6  0  0  0]]


You can see zeros at the end of the padded sentences.

Now we have everything that we need to create a sentiment classification model using word embeddings.

We will create a very simple text classification model with an embedding layer and no hidden layers. Look at the following script:

In [27]:
model = Sequential()
model.add(Embedding(vocab_length, 20, input_length=length_long_sentence))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In the script above, we create a Sequential model and add the Embedding layer as the first layer to the model. The length of the vocabulary is specified by the vocab_length parameter. The dimension of each word vector will be 20 and the input_length will be the length of the longest sentence, which is 7. Next, the Embedding layer is flattened so that it can be directly used with the densely connected layer. Since it is a binary classification problem, we use the sigmoid function as the loss function at the dense layer.

Next, we will compile the model and print the summary of our model, as shown below:

In [28]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7, 20)             1000      
_________________________________________________________________
flatten_1 (Flatten)          (None, 140)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 141       
Total params: 1,141
Trainable params: 1,141
Non-trainable params: 0
_________________________________________________________________
None


You can see that the first layer has 1000 trainable parameters. This is because our vocabulary size is 50 and each word will be presented as a 20 dimensional vector. Hence the total number of trainable parameters will be 1000. Similarly, the output from the embedding layer will be a sentence with 7 words where each word is represented by a 20 dimensional vector. However, when the 2D output is flattened, we get a 140 dimensional vector (7 x 20). The flattened vector is directly connected to the dense layer that contains 1 neuran.

Now let's train the model on our data using the fit method, as shown below:

In [29]:
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 4ms/step - loss: 0.6945 - acc: 0.4375
Epoch 2/100
1/1 [==============================] - 0s 725us/step - loss: 0.6912 - acc: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 767us/step - loss: 0.6880 - acc: 0.6250
Epoch 4/100
1/1 [==============================] - 0s 932us/step - loss: 0.6847 - acc: 0.7500
Epoch 5/100
1/1 [==============================] - 0s 709us/step - loss: 0.6815 - acc: 0.7500
Epoch 6/100
1/1 [==============================] - 0s 811us/step - loss: 0.6782 - acc: 0.7500
Epoch 7/100
1/1 [==============================] - 0s 889us/step - loss: 0.6750 - acc: 0.7500
Epoch 8/100
1/1 [==============================] - 0s 1ms/step - loss: 0.6718 - acc: 0.8750
Epoch 9/100
1/1 [==============================] - 0s 890us/step - loss: 0.6686 - acc: 0.8750
Epoch 10/100
1/1 [==============================] - 0s 1ms/step - loss: 0.6653 - acc: 0.9375
Epoch 11/100
1/1 [==============================] - 0s 969us/step

The model will be trained for 100 epochs.

We will train and test the model using the same corpus. Execute the following script to evaluate the model performance on our corpus:

In [30]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


In the output, you will see that model accuracy is 1.00 i.e. 100 percent.

Note: In real world applications, train and test sets should be different. We will see an example of that when we perform text classification on some real world data in an upcoming article.